In [1]:
import json, requests, os
import io
GPT_KEY = os.environ.get('OPENAI_API_KEY_MIT')
assert GPT_KEY, "Please set the OPENAI_API_KEY_MIT environment variable"
API_ROOT = "http://localhost:8000/"

In [2]:
import sqlalchemy as sql
eng = sql.create_engine('sqlite:///cache.db')

In [4]:
## create table in cache.db with columns file (string), data (json), and time (datetime)
# with eng.connect() as con:
#     con.execute(sql.text('''
#         CREATE TABLE IF NOT EXISTS cache (
#             file TEXT PRIMARY KEY,
#             data TEXT,
#             time TEXT
#         )
#     '''))

In [27]:
from pathlib import Path

def skema_integrated_pdf_extractions(pdf_path):
    if isinstance(pdf_path, str):
        pdf_path = Path(pdf_path)

    pdf_path : Path = pdf_path.expanduser()
    assert pdf_path.exists()

    output_path = pdf_path.with_suffix('.skema_integrated.json')
    if output_path.exists():
        with open(output_path) as f:
            return json.load(f)

    URL= "https://api.askem.lum.ai"

    params = {
       "annotate_skema":True,
       "annotate_mit": True,
    }

    files = [("pdfs", (str(pdf_path), open(str(pdf_path), "rb")))]
    response = requests.post(f"{URL}/text-reading/integrated-pdf-extractions", params=params, files=files)
    if response.status_code == 200:
        data = response.json()
        with open(output_path, 'w') as f:
            json.dump(data, f, indent=2)
        return data
    else:
        raise  Exception(f'{response.status_code=} {response.text=}')

In [17]:
import pathlib
data = json.load(open('/Users/orm/Downloads/response_1709051355014.json', 'r'))
path = pathlib.Path('/Users/orm/example_data/Patty-reviewed-1-28-24-Why is it difficult to accurately predict the COVID-19 epidemic (1).pdf')
json.dump(data, path.with_suffix('.skema_integrated.json').open('w'), indent=2)

In [12]:
import pathlib


In [62]:
from pathlib import Path
Path('foo.pad').with_suffix('.pdf')

PosixPath('foo.pdf')

In [57]:
import importlib
import mitaskem.api.routers.annotation
importlib.reload(mitaskem.api.routers.annotation)
from mitaskem.api.routers.annotation import list_scenarios_local



In [58]:
from pathlib import Path
example_data = Path("~/example_data/").expanduser()
for file in example_data.iterdir():
    if file.suffix == ".pdf":
        print(file)
        try:
            data = skema_integrated_pdf_extractions(file)
            df = list_scenarios_local(None, extractions=data, return_early=True)
            print(df)
        except Exception as e:
            print(e)


/Users/orm/example_data/Patty-reviewed-1-28-24-Why is it difficult to accurately predict the COVID-19 epidemic (1).pdf
    location             id name  value  \
56     Wuhan   E:-999142283    R    2.0   
58     Wuhan  E:-1416661983    R    2.0   
108      AIC   E:1393547595    N   40.0   
109      AIC  E:-2085445646    N   40.0   

                                               passage  
56   At the time of this manuscript , the consensus...  
58   At the time of this manuscript , the consensus...  
108  This definition should be used when K < N = 40...  
109  This definition should be used when K < N = 40...  
/Users/orm/example_data/Patty-reviewed-1-28-24-bertozzi-et-al-2020-the-challenges-of-modeling-and-forecasting-the-spread-of-covid-19.pdf
                                              location             id  \
36                                           Arlington    R:889375648   
37                                         Los Angeles    R:889375648   
38                      

In [61]:
import jmespath as jm

In [67]:
# class PaperVariables:


In [59]:
data

{'outputs': [{'data': {'attributes': [{'type': 'anchored_entity',
      'amr_element_id': None,
      'payload': {'id': {'id': 'E:-1080567600'},
       'mentions': [{'id': {'id': 'T:527526835'},
         'name': 'Wuhan',
         'extraction_source': {'page': 1,
          'block': 11,
          'surrounding_passage': 'Coronavirus disease 2019 ( COVID-19 ) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 ( SARSCoV-2 ) .\nThe disease was first identified December 2019 in Wuhan , the capital of Hubei , China , and has since spread globally , resulting in the ongoing 2020 pandemic outbreak [ 5 ] .\nThe COVID19 pandemic is considered as the biggest global threat worldwide because of thousands of confirmed infections , accompanied by thousands deaths over the world .',
          'char_start': 554,
          'char_end': 559,
          'document_reference': {'id': 'Patty-reviewed-1-28-24-Mathematical modeling of COVID-19 transmission dynamics with a case study

In [29]:
# data = pathlib.Path('/Users/orm/Downloads/Patty-reviewed-1-28-24-bertozzi-et-al-2020-the-challenges-of-modeling-and-forecasting-the-spread-of-covid-19.json')
# data = json.load(data.open('r'))

# have_path = pathlib.Path('/Users/orm/example_data/Patty-reviewed-1-28-24-bertozzi-et-al-2020-the-challenges-of-modeling-and-forecasting-the-spread-of-covid-19.json')
# want_path = have_path.with_suffix('.skema_integrated.json')

In [69]:
import PyPDF2

NotADirectoryError: [Errno 20] Not a directory: '/Users/orm/example_data/Patty-reviewed-1-28-24-Why is it difficult to accurately predict the COVID-19 epidemic (1).pdf/Users/orm/example_data/Patty-reviewed-1-28-24-bertozzi-et-al-2020-the-challenges-of-modeling-and-forecasting-the-spread-of-covid-19.pdf'

"Scenarios for the impact of short-term social distancing : fraction of population vs. date .
( Left ) California SIR model based on mortality data with parameters from Table 1 ( R0 = 2.7 , gamma = .12 , I0 = .1 ) under two scenarios : R0 constant in time ( light blue ) and R0 cut in half from 27 March ( 1 wk from the start of the California shutdown ) to 5 May but then returned to its original value , to represent a short-term distancing strategy ( dark blue ) .
( Right ) New York SIR model with parameters from Table 1 ( R0 = 4.1 , gamma = .1 , I0 = 05 ) under the same two scenarios but with short-term distancing occurring over the dates of 30 March ( 1 wk from the start of the New York shutdown ) to 5 May ."

[
{'location':'California', 'variable':'R0', 'value':2.7 },
{'location':'California', 'variable':'gamma', 'value':.12},
{'location':'California', 'variable':'I0', 'value':.1},

{'location':'California', 'variable':'R0', 'value':1.35 },
{'location':'California', 'variable':'gamma', 'value':.12},
{'location':'California', 'variable':'I0', 'value':.1},

{'location':'New York', 'variable':'R0', 'value':4.1 },
{'location':'New York', 'variable':'gamma', 'value':.1},
{'location':'New York', 'variable':'I0', 'value':.05},

{'location':'New York', 'variable':'R0', 'value':2.05 },
{'location':'New York', 'variable':'gamma', 'value':.1},
{'location':'New York', 'variable':'I0', 'value':.05},

]



In [53]:
paths = [
    '/Users/orm/example_data/Patty-reviewed-1-28-24-Why is it difficult to accurately predict the COVID-19 epidemic (1).pdf'
    '/Users/orm/example_data/Patty-reviewed-1-28-24-bertozzi-et-al-2020-the-challenges-of-modeling-and-forecasting-the-spread-of-covid-19.pdf',
    '/Users/orm/example_data/Patty-reviewed-1-28-24-Mathematical modeling of COVID-19 transmission dynamics with a case study of Wuhan.pdf'
]

for path in paths:
    data = skema_integrated_pdf_extractions(path)


AssertionError: 

In [37]:
df

,location,id,name,value,passage
0,capital,E:-1080567600,Wuhan,None,Coronavirus disease 2019 ( COVID-19 ) is an in...
1,capital,E:-98761542,Hubei,None,Coronavirus disease 2019 ( COVID-19 ) is an in...
2,Wuhan,R:-1200572007,equi,None,"In Section 2 , we propose a new model for COVI..."
3,Wuhan,R:2065831243,R,None,"In Section 2 , we propose a new model for COVI..."
4,Wuhan,E:-1549289716,R,None,A qualitative analysis of the model is investi...
...,...,...,...,...,...
216,J.J,R:1441106003,h,None,It can be understood as the average number of ...
217,Solitons,R:1441106003,h,None,It can be understood as the average number of ...
218,J.J,E:30400911,below generation matrices F,0,It can be understood as the average number of ...
219,Solitons,E:30400911,below generation matrices F,0,It can be understood as the average number of ...


In [45]:
df = df.sort_values('location')
df = df[~df.value.isna()]

In [50]:
for passage in df.passage.unique():
    print(passage)
    print('----')

A qualitative analysis of the model is investigated in Section 3 : in Section 3.1 , we compute the of the COVID-19 system model ; in basic reproduction number R 0 Section 3.2 , we study the local stability of the disease free equi .
The sensitivity of the basic reproduction librium in terms of R 0 with respect to the parameters of the system model is number R 0 given in Section 4 .
The usefulness of our model is then illustrated in Section 5 of numerical simulations , where we use real data from Wuhan .
----
It can be understood as the average number of cases one infected individual generates , over the course of its infectious period , in an otherwise uninfected population .
Using the next generation matrix approach outlined in van den Driessche and Watmough [ 13 ] to our model ( 1 ) , the basic reproduction number can be computed by considering the below generation matrices F and V , that is , the Jacobian matrices associated to the rate of appearance of new infections and the net ra

In [34]:
df

,varname,value,geo
0,L C ( 0 ),number of confirmed cases 04 January 2020,None
1,L C ( 65 ),number of confirmed cases 09 March 2020,None
2,L D ( 0 ),number of dead on January 4,None
3,L D ( 65 ),"number of dead on March 9, 2020",None


In [31]:
df

,varname,value,geo
0,basic reproduction number R 0,2,Wuhan
1,K,< N,None
2,N,40,None
3,K,> N,None
4,N,40,None


In [139]:
df.drop_duplicates()

,varname,value,geo
0,R ( t ),1.00,China
1,R ( t ),1.00,Italy
2,reproduction number,2.50,United States
3,B ( CID :88 ) R ( T ),3.00,IK
4,K,1.00,None
5,R0,2.70,California
6,gamma,0.12,California
7,I0,0.10,California
8,R0,4.10,New York
9,gamma,0.10,New York


In [55]:
pd.DataFrame(list_scenarios_local(None, extractions=data, flatten_groups=True)and)

.to_csv('~/bertozzi_scenarios.csv')

In [41]:
import jmespath as jp
import pandas as pd

var_extractor_expr = """
        outputs[0].data.attributes[? type == 'anchored_entity'][].payload
            .{ id:id.id,
               name:mentions[0].name
               passage:mentions[0].extraction_source.surrounding_passage
               value:value_descriptions[0].value.amount
            }
    """

# var_extractor_expr = """
#         outputs[0].data.attributes[? type == 'anchored_entity'][].payload
# """

ans = jp.search(var_extractor_expr.replace('\n', ' '), data)
ans

[{'id': 'R:1407870398',
  'name': 'dDepartment of Computer Science',
  'passage': 'aDepartment of Mathematics , University of California , Los Angeles , ca 90095 ; bDepartment of Mechanical and Aerospace Engineering , University of California , Los Angeles , Ca 90095 ; cDepartment of Bioengineering , University of California , Los Angeles , ca 90095 ; dDepartment of Computer Science , Indiana University-Purdue University Indianapolis , Indianapolis , in 46202 ; eDepartment of Mathematics , Georgia Institute of Technology , Atlanta , GA 30332 ; and fDepartment of Political Science , University of Texas at Arlington , Arlington , Tx 76019 .',
  'value': '90095'},
 {'id': 'R:-2133161548',
  'name': 'fDepartment of Political Science',
  'passage': 'aDepartment of Mathematics , University of California , Los Angeles , ca 90095 ; bDepartment of Mechanical and Aerospace Engineering , University of California , Los Angeles , Ca 90095 ; cDepartment of Bioengineering , University of California ,

In [84]:
var_extractor_expr = "attributes[? type == 'anchored_entity'].payload"
jp.search(var_extractor_expr, extractions)[0].keys()

dict_keys(['id', 'mentions', 'text_descriptions', 'value_descriptions', 'groundings', 'data_columns'])

In [100]:
## send post request to
params = {
    "gpt_key": GPT_KEY,
}

files = [("extractions_file", ('', io.BytesIO(json.dumps(data).encode('utf-8'))))]

response = requests.post('http://localhost:8000/annotation/list_scenarios/', params=params, files=files)
response.status_code, print(response.text)

[{"location":"17 March 2020","entities":[{"id":"R:1330896514","names":["NPIs"],"values":[]}]},{"location":"AIC","entities":[{"id":"E:-27420438","names":["a"],"values":[]},{"id":"E:1221701905","names":["SIR"],"values":[]},{"id":"E:-529909366","names":["SIR"],"values":[]},{"id":"E:1242757458","names":["SIR"],"values":[]},{"id":"R:1326130415","names":["AIC"],"values":[]},{"id":"E:587018829","names":["AICmin"],"values":["2"]},{"id":"R:166630294","names":["AIC"],"values":[]},{"id":"E:639721417","names":["CA"],"values":[]},{"id":"R:-418568586","names":["NY"],"values":[]},{"id":"R:1826066952","names":["k"],"values":[]},{"id":"R:719599970","names":["b"],"values":[]}]},{"location":"Aerospace Engineering","entities":[{"id":"R:1407870398","names":["dDepartment of Computer Science"],"values":["90095"]},{"id":"R:-2133161548","names":["fDepartment of Political Science"],"values":["30332"]},{"id":"E:-443623759","names":["Tx"],"values":[]},{"id":"R:889375648","names":["Mechanical"],"values":["90095"]}

(200, None)

In [101]:
response.json()

[{'location': '17 March 2020',
  'entities': [{'id': 'R:1330896514', 'names': ['NPIs'], 'values': []}]},
 {'location': 'AIC',
  'entities': [{'id': 'E:-27420438', 'names': ['a'], 'values': []},
   {'id': 'E:1221701905', 'names': ['SIR'], 'values': []},
   {'id': 'E:-529909366', 'names': ['SIR'], 'values': []},
   {'id': 'E:1242757458', 'names': ['SIR'], 'values': []},
   {'id': 'R:1326130415', 'names': ['AIC'], 'values': []},
   {'id': 'E:587018829', 'names': ['AICmin'], 'values': ['2']},
   {'id': 'R:166630294', 'names': ['AIC'], 'values': []},
   {'id': 'E:639721417', 'names': ['CA'], 'values': []},
   {'id': 'R:-418568586', 'names': ['NY'], 'values': []},
   {'id': 'R:1826066952', 'names': ['k'], 'values': []},
   {'id': 'R:719599970', 'names': ['b'], 'values': []}]},
 {'location': 'Aerospace Engineering',
  'entities': [{'id': 'R:1407870398',
    'names': ['dDepartment of Computer Science'],
    'values': ['90095']},
   {'id': 'R:-2133161548',
    'names': ['fDepartment of Politica